# **Space Invaders RL**

## **Load and Install Dependencies**

In [2]:
import gym
import random

In [3]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [4]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [6]:
eps = 5
for episode in range(eps):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = random.choice([0,1,2,3,4,5])
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score: {}'.format(episode, score))
env.close()

Episode:0 Score: 55.0
Episode:1 Score: 230.0
Episode:2 Score: 65.0
Episode:3 Score: 215.0
Episode:4 Score: 415.0


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [6]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32,(8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Convolution2D(64,(4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64,(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [7]:
model = build_model(height, width, channels, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

In [8]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [9]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg',
                  nb_actions=actions, nb_steps_warmup=10000)
    return dqn

In [11]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

Training for 10000 steps ...
 1155/10000: episode: 1, duration: 307.809s, episode steps: 1155, steps per second:   4, episode reward: 415.000, mean reward:  0.359 [ 0.000, 200.000], mean action: 2.559 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 2025/10000: episode: 2, duration: 227.855s, episode steps: 870, steps per second:   4, episode reward: 195.000, mean reward:  0.224 [ 0.000, 30.000], mean action: 2.828 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 2421/10000: episode: 3, duration: 104.116s, episode steps: 396, steps per second:   4, episode reward: 60.000, mean reward:  0.152 [ 0.000, 15.000], mean action: 2.750 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 2765/10000: episode: 4, duration: 91.659s, episode steps: 344, steps per second:   4, episode reward: 20.000, mean reward:  0.058 [ 0.000, 10.000], mean action: 2.881 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 3645/10000: episode: 5, duration: 418.079s, episode steps: 880, steps per second:  

In [12]:
scores =dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\user\anaconda3\envs\courseenv\lib\site-packages\gym\envs\atari\environment.py:269: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  "We strongly suggest supplying `render_mode` when "
C:\Users\user\anaconda3\envs\courseenv\lib\site-packages\pyglet\image\codecs\wic.py:434: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode 1: reward: 165.000, steps: 704
Episode 2: reward: 130.000, steps: 648
Episode 3: reward: 185.000, steps: 666
Episode 4: reward: 175.000, steps: 688
Episode 5: reward: 230.000, steps: 792
Episode 6: reward: 130.000, steps: 708
Episode 7: reward: 175.000, steps: 680
Episode 8: reward: 335.000, steps: 815
Episode 9: reward: 150.000, steps: 614
Episode 10: reward: 520.000, steps: 1081
219.5
